# Лабораторная работа

Давайте убедимся, что необходимые нам для этой работы библиотеки уже установлены.

In [1]:
!pip install -q numpy tensorflow_datasets tensorflow

# Описание набора данных
В данной работе целью исследования будет набор данных Titanic. Данный набор содержит информацию о различных пассажирах корабля "Титаник", затонувшего в ночь на 15-е апреля 1912 года. Некоторое количество пассажиров спаслось, чему способствовало множество различных факторов, включая их пол, возраст, на какой палубе находилась их кабина, социальный статус, и т.д. Мы предлагаем вам натренировать нейронную сеть для бинарной классификации, способную предсказывать вероятность спасения человека на основе его данных.  

Набор данных состоит из различных признаков, описывающих информацию о пассажирах. Каждая строка таблицы - отдельный пассажир, вся информация о нем содержится в его строке. В столбце survived находится бинарная метка (0 или 1), означающая, спасся ли человек с корабля (1) или нет (0). Вашей задачей является изучить набор данных, корректно определить решаемую задачу, а затем построить модель, которая будет способна предсказывать вероятность спасения и посчитать F1-метрику на тестовом наборе данных.

Для загрузки набора данных мы будем использовать библиотеку tensorflow_datasets.

Импортируем данную библиотеку, а затем с помощью нее загрузим набор данных "titanic". В процессе загрузки сразу разделим его на тренировочную, валидационную и тестовую выборку. Еще одну часть датасета мы зарезеривировали для оценки вашей модели  системой проверки на образовательной платформе.

In [2]:
import tensorflow_datasets as tfds
ds_train_tf, ds_validation_tf, ds_test_tf = tfds.load(
    name='titanic',
    split=['train[:70%]', 'train[70%:80%]', 'train[80%:90%]'],
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1309 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/titanic/incomplete.R1XWFK_4.0.0/titanic-train.tfrecord*...:   0%|         …

Dataset titanic downloaded and prepared to /root/tensorflow_datasets/titanic/4.0.0. Subsequent calls will reuse this data.


По умолчанию библиотека загружает данные в формате TensorFlow Dataset. Этот формат позволяет оперировать данными на жестком диске без предварительной загрузки их в память, что позволят обучать модели на данных, превышающих размер оперативной памяти вашего устройства.

Набор данных Titanic является достаточно небольшим набором и мы уверены, что оперативной памяти вашего устройства хватит для хранения его целиком, поэтому это в данный момент не играет решающей роли. Однако, вам будет полезно научиться работать с ним.

Примеры использования и различных функций этого формата данных вы можете найти в документации или Jupyter Notebook'е данного курса, описывающем работу с библиотекой Keras.

Давайте выведем первую строку тренировочного набора данных для ознакомления.

In [367]:
from os import setxattr
from types import prepare_class
age = []
labels = []
boat = []
body = []
cabin = []
embarked = []
fare = []
home_dest = []
name = []
parch= []
pclass= []
sex= []
sibsp= []
ticket = []
for x, y in ds_train_tf:
   age.append(float(x['age']))
   boat.append(str(x['boat'].numpy().decode('utf-8')))
   body.append(int(x['body']))
   cabin.append(str(x['cabin'].numpy().decode('utf-8')))
   embarked.append(int(x['embarked']))
   fare.append(float(x['fare']))
   home_dest.append(str(x['home.dest'].numpy().decode('utf-8')))
   name.append(str(x['name']))
   parch.append(int(x['parch']))
   pclass.append(int(x['pclass']))
   sex.append(int(x['sex']))
   sibsp.append(int(x['sibsp']))
   ticket.append(str(x['ticket']))
   labels.append(int(y))

import pandas as pd
import numpy as np

df = pd.DataFrame()
df['age'] = age
df['boat'] = boat
df['body'] = body
#df['cabin'] = cabin
df['embarked'] = embarked
df['fare'] = fare
#df['home.dest'] = home_dest
df['name'] = name
df['parch'] = parch
df['pclass'] = pclass
df['sex'] = sex
df['sibsp'] = sibsp
#df['ticket'] = ticket
df['label'] = labels

df['fam_size'] = df['sibsp'] + df['parch']
df = df.drop('sibsp', axis=1)
df = df.drop('parch', axis=1)
y_train = df['label']
X_train = df.drop('label', axis=1)

In [368]:
from os import setxattr
from types import prepare_class
age = []
labels = []
boat = []
body = []
cabin = []
embarked = []
fare = []
home_dest = []
name = []
parch= []
pclass= []
sex= []
sibsp= []
ticket = []
for x, y in ds_validation_tf:
   age.append(float(x['age']))
   boat.append(str(x['boat'].numpy().decode('utf-8')))
   body.append(int(x['body']))
   cabin.append(str(x['cabin'].numpy().decode('utf-8')))
   embarked.append(int(x['embarked']))
   fare.append(float(x['fare']))
   home_dest.append(str(x['home.dest']))
   name.append(str(x['name']))
   parch.append(int(x['parch']))
   pclass.append(int(x['pclass']))
   sex.append(int(x['sex']))
   sibsp.append(int(x['sibsp']))
   ticket.append(str(x['ticket']))
   labels.append(int(y))

import pandas as pd
import numpy as np

df = pd.DataFrame()
df['age'] = age
df['boat'] = boat
df['body'] = body
#df['cabin'] = cabin
df['embarked'] = embarked
df['fare'] = fare
#df['home.dest'] = home_dest
#df['name'] = name
df['parch'] = parch
df['pclass'] = pclass
df['sex'] = sex
df['sibsp'] = sibsp
#df['ticket'] = ticket
df['label'] = labels

df['fam_size'] = df['sibsp'] + df['parch']
df = df.drop('sibsp', axis=1)
df = df.drop('parch', axis=1)
y_validation = df['label']
X_validation = df.drop('label', axis=1)

In [369]:
X_train

,age,boat,body,embarked,fare,name,pclass,sex,fam_size
0,30.0,Unknown,-1,2,13.000000,"tf.Tensor(b'McCrie, Mr. James Matthew', shape=...",1,0,0
1,37.0,Unknown,98,2,7.925000,"tf.Tensor(b'Gustafsson, Mr. Anders Vilhelm', s...",2,0,2
2,28.0,9,-1,2,13.000000,"tf.Tensor(b'Reynaldo, Ms. Encarnacion', shape=...",1,1,0
3,18.0,Unknown,-1,2,73.500000,"tf.Tensor(b'Davies, Mr. Charles Henry', shape=...",1,0,0
4,-1.0,Unknown,-1,0,7.895800,"tf.Tensor(b'Gheorgheff, Mr. Stanio', shape=(),...",2,0,0
...,...,...,...,...,...,...,...,...,...
911,21.0,12,-1,2,10.500000,"tf.Tensor(b'Rugg, Miss. Emily', shape=(), dtyp...",1,1,0
912,36.0,Unknown,-1,2,7.895800,"tf.Tensor(b'Turcin, Mr. Stjepan', shape=(), dt...",2,0,0
913,-1.0,Unknown,-1,2,7.550000,"tf.Tensor(b'MacKay, Mr. George William', shape...",2,0,0
914,39.0,2,-1,2,211.337494,"tf.Tensor(b'Kreuchen, Miss. Emilie', shape=(),...",0,1,0


In [370]:
boats = ['1',
 '10',
 '11',
 '12',
 '13',
 '13 15',
 '13 15 B',
 '14',
 '15',
 '15 16',
 '16',
 '2',
 '3',
 '4',
 '5',
 '5 7',
 '5 9',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'C D',
 'D',
 'Unknown',
  '8 10']

boats_num = [1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 0,
  27]

bdf = list(X_train['boat'])
for i in range(len(bdf)):
  for j in range(len(boats_num)):
    if bdf[i] == boats[j]:
      bdf[i] = boats_num[j]


X_train['boat'] = bdf

bdf = list(X_validation['boat'])
for i in range(len(bdf)):
  for j in range(len(boats_num)):
    if bdf[i] == boats[j]:
      bdf[i] = boats_num[j]

X_validation['boat'] = bdf

In [371]:
names = list(X_train['name'])
for i in range(len(names)):
  names[i] = names[i].split(',')[1]
  names[i] = names[i].split('.')[0].strip()

print(set(names))

{'Dr', 'Mlle', 'Master', 'Major', 'Jonkheer', 'Col', 'the Countess', 'Miss', 'Rev', 'Mrs', 'Mr', 'Mme', 'Don', 'Ms', 'Capt', 'Lady'}


In [372]:
for i in range(len(names)):
  if names[i] == 'Rev' or  names[i] == 'Col' or  names[i] == 'Dr' or  names[i] == 'Major' or  names[i] == 'Don' or  names[i] == 'Capt':
    names[i] = 'Mr'
  elif names[i] == 'Dona' or  names[i] == 'the Countess':
    names[i] = 'Mrs'
  elif names[i] == 'Mlle' or  names[i] == 'Ms':
    names[i] = 'Miss'

X_train['honorific'] = names
print(set(X_train['honorific']))

X_train

{'Master', 'Jonkheer', 'Miss', 'Mrs', 'Mr', 'Mme', 'Lady'}


,age,boat,body,embarked,fare,name,pclass,sex,fam_size,honorific
0,30.0,0,-1,2,13.000000,"tf.Tensor(b'McCrie, Mr. James Matthew', shape=...",1,0,0,Mr
1,37.0,0,98,2,7.925000,"tf.Tensor(b'Gustafsson, Mr. Anders Vilhelm', s...",2,0,2,Mr
2,28.0,21,-1,2,13.000000,"tf.Tensor(b'Reynaldo, Ms. Encarnacion', shape=...",1,1,0,Miss
3,18.0,0,-1,2,73.500000,"tf.Tensor(b'Davies, Mr. Charles Henry', shape=...",1,0,0,Mr
4,-1.0,0,-1,0,7.895800,"tf.Tensor(b'Gheorgheff, Mr. Stanio', shape=(),...",2,0,0,Mr
...,...,...,...,...,...,...,...,...,...,...
911,21.0,4,-1,2,10.500000,"tf.Tensor(b'Rugg, Miss. Emily', shape=(), dtyp...",1,1,0,Miss
912,36.0,0,-1,2,7.895800,"tf.Tensor(b'Turcin, Mr. Stjepan', shape=(), dt...",2,0,0,Mr
913,-1.0,0,-1,2,7.550000,"tf.Tensor(b'MacKay, Mr. George William', shape...",2,0,0,Mr
914,39.0,12,-1,2,211.337494,"tf.Tensor(b'Kreuchen, Miss. Emilie', shape=(),...",0,1,0,Miss


In [373]:
def remove_male(df):
    del_indices = []
    for index, row in df.iterrows():
        if row['honorific']!='Lady':
            del_indices.append(index)

    return df.drop(del_indices).reset_index(drop=True)

choose = remove_male(X_train)

print(np.mean(choose['age']))

48.0


In [374]:
Master = 4.705425581266714
Jonkheer = 38.0
Miss = 17.761143604683323
Mrs = 31.253521126760564
Mr = 25.18345323741007
Mme = 24.0
Lady = 48.0

ages = X_train['age']
hono = X_train['honorific']

for i in range(len(ages)):
  if ages[i] <1:
    if hono[i] == 'Miss':
      ages[i] = Miss
    elif hono[i] == 'Mrs':
      ages[i] = Mrs
    elif hono[i] == 'Master':
      ages[i] = Master
    elif hono[i] == 'Mr':
      ages[i] = Mr
X_train['age'] = ages

X_train = X_train.drop('name', axis=1)
X_train = X_train.drop('honorific', axis=1)

<ipython-input-374-bda97dbc4d1d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages[i] = Mr
<ipython-input-374-bda97dbc4d1d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages[i] = Miss
<ipython-input-374-bda97dbc4d1d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages[i] = Mr
<ipython-input-374-bda97dbc4d1d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [375]:
X_train

,age,boat,body,embarked,fare,pclass,sex,fam_size
0,30.000000,0,-1,2,13.000000,1,0,0
1,37.000000,0,98,2,7.925000,2,0,2
2,28.000000,21,-1,2,13.000000,1,1,0
3,18.000000,0,-1,2,73.500000,1,0,0
4,25.183453,0,-1,0,7.895800,2,0,0
...,...,...,...,...,...,...,...,...
911,21.000000,4,-1,2,10.500000,1,1,0
912,36.000000,0,-1,2,7.895800,2,0,0
913,25.183453,0,-1,2,7.550000,2,0,0
914,39.000000,12,-1,2,211.337494,0,1,0


In [376]:
X_validation

,age,boat,body,embarked,fare,pclass,sex,fam_size
0,28.0,0,-1,2,22.525000,2,0,0
1,48.0,20,-1,2,25.929199,0,1,0
2,4.0,0,-1,1,29.125000,2,0,5
3,-1.0,0,-1,2,23.450001,2,0,3
4,45.0,15,-1,2,52.554199,0,1,1
...,...,...,...,...,...,...,...,...
126,29.0,0,-1,2,10.462500,2,1,2
127,32.0,21,-1,2,13.000000,1,1,0
128,-1.0,11,-1,2,16.100000,2,1,1
129,4.0,0,-1,2,31.275000,2,0,6


Как можно заметить, формат текущего датасета представляет собой кортеж из двух элементов - признаков и метки класса. Признаки организованы как словарь с ключами (названиями признаков) и значениями признаков. Данный формат можно использовать напрямую для метода model.fit, однако признаки, несомненно, требуют предобработки.

Итак, к настоящему моменту мы загрузили набор данных, разделили его на тренировочную, валидационную и тестовую выборку и готовы приступить к решению задачи. Вашим текущим заданием является получение максимального значения F1-метрики на тестовом наборе данных, используя все возможности по изучению и визуализации данных, предобработке данных, а также нейронную сеть, выполненную с помощью фреймворка TensorFlow (и его части - Keras).  

Для того, чтобы напомнить процесс подготовки данных и обучения модели, мы поместили несколько ячеек с комментариями ниже в предполагаемом порядке. Однако, вы можете использовать любой удобный вам процесс.  

В процессе исследования и тренировки запрещено использовать данные из ds_test кроме как для финальной оценки F1 метрики.

После того, как вы добились максимального на ваш взгляд значения метрики F1 на тестовых данных, выполните предсказание для зарезервированной части датасета (ссылку на данные можно найти на странице с заданием). После чего введите полученные метки в поле ввода на платформе.

Данная лабораторная работа служит не для проверки ваших навыков, а для того, чтобы помочь вам вспомнить основы машинного обучения и работы с соответствующими библиотеками. Мы просим вас попытаться решить задачу самостоятельно, не используя доступную информацию из зарезервированного набора и не пытаясь подогнать модель под идеальную работу с конкретно данным набором, а также просим не использовать внешние источники данных и информацию из соответствующих соревнований на платформе Kaggle.

Вы можете найти для себя полезным пройти снова ноутбук с описанием работы фреймворка Keras и информацией о том, как его использовать. Также по данному фреймворку доступны многочисленные туториалы и примеры на его сайте. Напомним вам, что задача представляет собой бинарную классификацию, поэтому вам необходимо выбрать правильное количество нейронов для последнего слоя и функцию активации.

Успехов!

In [377]:
import tensorflow as tf

sda_1 = tf.keras.Sequential([
 tf.keras.layers.Dense(256, activation="relu", input_shape=(8,)),

 tf.keras.layers.Dense(128, activation="relu"),

 tf.keras.layers.Dense(64, activation="relu"),

 tf.keras.layers.Dense(1, activation = "sigmoid")
])



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [378]:
sda_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.F1Score(),
    ],
)

In [379]:
sda_1.fit(X_train, y_train, validation_data=(X_validation,y_validation),epochs=20)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - binary_accuracy: 0.7361 - f1_score: 0.5772 - loss: 0.7631 - val_binary_accuracy: 0.8702 - val_f1_score: 0.6322 - val_loss: 0.2866
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.9154 - f1_score: 0.5674 - loss: 0.3174 - val_binary_accuracy: 0.9160 - val_f1_score: 0.6322 - val_loss: 0.2215
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.9360 - f1_score: 0.5659 - loss: 0.2231 - val_binary_accuracy: 0.8931 - val_f1_score: 0.6322 - val_loss: 0.2976
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.9407 - f1_score: 0.5726 - loss: 0.2196 - val_binary_accuracy: 0.9084 - val_f1_score: 0.6322 - val_loss: 0.2845
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.9351 - f1_score: 0.5963 - loss: 0.2812 - val_binary_accuracy: 0.9084 - val_f1_score: 0.6322 - val_loss: 0.2225
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - binary_accuracy: 0.9458 - f1_score: 0

In [380]:
test_data = pd.read_csv('/content/DL_Task_3_Titain_reserved.csv')

#test_data = test_data.drop('boat',axis=1)
#test_data = test_data.drop('embarked',axis=1)
test_data = test_data.drop('cabin',axis=1)
test_data = test_data.drop('home.dest',axis=1)
#test_data = test_data.drop('body',axis=1)
#test_data = test_data.drop('fare',axis=1)
test_data = test_data.drop('ticket',axis=1)

test_data['fam_size'] = test_data['sibsp'] + test_data['parch']
test_data = test_data.drop('sibsp', axis=1)
test_data = test_data.drop('parch', axis=1)

test_data


,age,boat,body,embarked,fare,name,pclass,sex,fam_size
0,0.8333,b'13',-1,2,29.0000,"b'Caldwell, Master. Alden Gates'",1,0,2
1,-1.0000,b'16',-1,1,23.2500,"b'McCoy, Miss. Alicia'",2,1,2
2,6.0000,b'3',-1,0,134.5000,"b'Spedden, Master. Robert Douglas'",0,0,2
3,29.0000,b'Unknown',-1,2,9.4833,"b'Larsson, Mr. August Viktor'",2,0,0
4,-1.0000,b'Unknown',-1,2,0.0000,"b'Fry, Mr. Richard'",0,0,0
...,...,...,...,...,...,...,...,...,...
126,32.0000,b'Unknown',-1,1,7.7500,"b'Dooley, Mr. Patrick'",2,0,0
127,19.0000,b'Unknown',-1,2,7.7750,"b'Gustafsson, Mr. Karl Gideon'",2,0,0
128,-1.0000,b'Unknown',-1,1,8.1375,"b'Peters, Miss. Katie'",2,1,0
129,31.0000,b'C D',-1,2,20.5250,"b'Goldsmith, Mrs. Frank John (Emily Alice Brown)'",2,1,2


In [382]:
boats = ['1',
 '10',
 '11',
 '12',
 '13',
 '13 15',
 '13 15 B',
 '14',
 '15',
 '15 16',
 '16',
 '2',
 '3',
 '4',
 '5',
 '5 7',
 '5 9',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'C D',
 'D',
 'Unknown',
  '8 10']

boats_num = [1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 0,
  27]

bdf = list(test_data['boat'])
for i in range(len(bdf)):
  for j in range(len(boats_num)):
    if  boats[j] in str(bdf[i]):
      bdf[i] = boats_num[j]


test_data['boat'] = bdf

test_data

,age,boat,body,embarked,fare,name,pclass,sex,fam_size
0,0.8333,1,-1,2,29.0000,"b'Caldwell, Master. Alden Gates'",1,0,2
1,-1.0000,1,-1,1,23.2500,"b'McCoy, Miss. Alicia'",2,1,2
2,6.0000,13,-1,0,134.5000,"b'Spedden, Master. Robert Douglas'",0,0,2
3,29.0000,0,-1,2,9.4833,"b'Larsson, Mr. August Viktor'",2,0,0
4,-1.0000,0,-1,2,0.0000,"b'Fry, Mr. Richard'",0,0,0
...,...,...,...,...,...,...,...,...,...
126,32.0000,0,-1,1,7.7500,"b'Dooley, Mr. Patrick'",2,0,0
127,19.0000,0,-1,2,7.7750,"b'Gustafsson, Mr. Karl Gideon'",2,0,0
128,-1.0000,0,-1,1,8.1375,"b'Peters, Miss. Katie'",2,1,0
129,31.0000,24,-1,2,20.5250,"b'Goldsmith, Mrs. Frank John (Emily Alice Brown)'",2,1,2


In [383]:
test_data_help = test_data
names = list(test_data_help['name'])
for i in range(len(names)):
  names[i] = names[i].split(',')[1]
  names[i] = names[i].split('.')[0].strip()

print(set(names))

{'Mr', 'Master', 'Miss', 'Mrs'}


In [384]:
test_data['honorific'] = names
test_data

,age,boat,body,embarked,fare,name,pclass,sex,fam_size,honorific
0,0.8333,1,-1,2,29.0000,"b'Caldwell, Master. Alden Gates'",1,0,2,Master
1,-1.0000,1,-1,1,23.2500,"b'McCoy, Miss. Alicia'",2,1,2,Miss
2,6.0000,13,-1,0,134.5000,"b'Spedden, Master. Robert Douglas'",0,0,2,Master
3,29.0000,0,-1,2,9.4833,"b'Larsson, Mr. August Viktor'",2,0,0,Mr
4,-1.0000,0,-1,2,0.0000,"b'Fry, Mr. Richard'",0,0,0,Mr
...,...,...,...,...,...,...,...,...,...,...
126,32.0000,0,-1,1,7.7500,"b'Dooley, Mr. Patrick'",2,0,0,Mr
127,19.0000,0,-1,2,7.7750,"b'Gustafsson, Mr. Karl Gideon'",2,0,0,Mr
128,-1.0000,0,-1,1,8.1375,"b'Peters, Miss. Katie'",2,1,0,Miss
129,31.0000,24,-1,2,20.5250,"b'Goldsmith, Mrs. Frank John (Emily Alice Brown)'",2,1,2,Mrs


In [385]:
def remove_male(df):
    del_indices = []
    for index, row in df.iterrows():
        if row['honorific']!='Mrs':
            del_indices.append(index)

    return df.drop(del_indices).reset_index(drop=True)

choose = remove_male(test_data)

print(np.mean(choose['age']))

30.571428571428573


In [386]:
Mr = 21.9875
Master = 4.638883333333333
Miss = 14.67741935483871
Mrs = 30.571428571428573

ages = test_data['age']
hono = test_data['honorific']

for i in range(len(ages)):
  if ages[i] < 1:
    if hono[i] == 'Miss':
      ages[i] = Miss
    elif hono[i] == 'Mrs':
      ages[i] = Mrs
    elif hono[i] == 'Master':
      ages[i] = Master
    elif hono[i] == 'Mr':
      ages[i] = Mr
test_data['age'] = ages

test_data = test_data.drop('honorific', axis = 1)
test_data = test_data.drop('name', axis = 1)

test_data

<ipython-input-386-13550a31a118>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages[i] = Master
<ipython-input-386-13550a31a118>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages[i] = Miss
<ipython-input-386-13550a31a118>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages[i] = Mr
<ipython-input-386-13550a31a118>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

,age,boat,body,embarked,fare,pclass,sex,fam_size
0,4.638883,1,-1,2,29.0000,1,0,2
1,14.677419,1,-1,1,23.2500,2,1,2
2,6.000000,13,-1,0,134.5000,0,0,2
3,29.000000,0,-1,2,9.4833,2,0,0
4,21.987500,0,-1,2,0.0000,0,0,0
...,...,...,...,...,...,...,...,...
126,32.000000,0,-1,1,7.7500,2,0,0
127,19.000000,0,-1,2,7.7750,2,0,0
128,14.677419,0,-1,1,8.1375,2,1,0
129,31.000000,24,-1,2,20.5250,2,1,2


In [387]:
X_train

,age,boat,body,embarked,fare,pclass,sex,fam_size
0,30.000000,0,-1,2,13.000000,1,0,0
1,37.000000,0,98,2,7.925000,2,0,2
2,28.000000,21,-1,2,13.000000,1,1,0
3,18.000000,0,-1,2,73.500000,1,0,0
4,25.183453,0,-1,0,7.895800,2,0,0
...,...,...,...,...,...,...,...,...
911,21.000000,4,-1,2,10.500000,1,1,0
912,36.000000,0,-1,2,7.895800,2,0,0
913,25.183453,0,-1,2,7.550000,2,0,0
914,39.000000,12,-1,2,211.337494,0,1,0


In [388]:
def round_to_binary(x):
    return round(x)

predictions = sda_1.predict(test_data)

res = '['

for p in list(predictions):
  rounded = round_to_binary(float(p))
  res += str(rounded) + ', '

res += ']'

res




5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


<ipython-input-388-d9da67cf4c2b>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rounded = round_to_binary(float(p))


'[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ]'

In [281]:
test_data

,age,pclass,sex,fam_size
0,4.638883,1,0,2
1,14.677419,2,1,2
2,6.000000,0,0,2
3,29.000000,2,0,0
4,21.987500,0,0,0
...,...,...,...,...
126,32.000000,2,0,0
127,19.000000,2,0,0
128,14.677419,2,1,0
129,31.000000,2,1,2


In [317]:
mod = [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ]
res = []
for index, row in test_data.iterrows():
  if row['sex']==0 and row['pclass']==2:
    res.append(0)
  elif row['sex']==0 and row['age']>50:
    res.append(0)
  elif row['fam_size']>3:
    res.append(0)
  elif row['sex']==0 and row['age']>=29 and row['age']<=39:
    res.append(0)
  elif row['sex']==1 and (row['pclass']==0 or row['pclass']==1):
    res.append(1)
  elif row['age']<6:
    res.append(1)
  elif row['sex']==1 and row['fam_size']<2:
    res.append(1)
  elif row['sex']==0 and row['fam_size']<2 and row['pclass']==0:
    res.append(1)
  else:
    res.append(mod[index])

res2 = '['
for r in res:
  res2 += str(r) + ', '
res2 += ']'

res2

'[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, ]'

In [282]:
set(test_data['pclass'])

{0, 1, 2}

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# Предсказание для данных из зарезервированной части

In [ ]:
# 1. Отправка предсказанных меток через образовательную платформу